### 实现vibe算法：视频流实时场景抑制+动态目标检测

待调整


In [ ]:
import cv2
import numpy as np

class ViBe:
    def __init__(self, num_samples=20, min_matches=2, radius=20, subsample_factor=16):
        self.num_samples = num_samples
        self.min_matches = min_matches
        self.radius = radius
        self.subsample_factor = subsample_factor
        self.samples = None

    def initialize(self, frame):
        height, width = frame.shape[:2]
        self.samples = np.zeros((height, width, self.num_samples), dtype=np.uint8)
        for i in range(height):
            for j in range(width):
                for k in range(self.num_samples):
                    rand_i = np.clip(i + np.random.randint(-1, 2), 0, height - 1)
                    rand_j = np.clip(j + np.random.randint(-1, 2), 0, width - 1)
                    self.samples[i, j, k] = frame[rand_i, rand_j]

    def update(self, frame):
        height, width = frame.shape[:2]
        fg_mask = np.zeros((height, width), dtype=np.uint8)
        for i in range(height):
            for j in range(width):
                count = 0
                for k in range(self.num_samples):
                    if abs(int(frame[i, j]) - int(self.samples[i, j, k])) < self.radius:
                        count += 1
                        if count >= self.min_matches:
                            break
                if count < self.min_matches:
                    fg_mask[i, j] = 255
                if np.random.randint(0, self.subsample_factor) == 0:
                    rand_sample = np.random.randint(0, self.num_samples)
                    self.samples[i, j, rand_sample] = frame[i, j]
                    rand_i = np.clip(i + np.random.randint(-1, 2), 0, height - 1)
                    rand_j = np.clip(j + np.random.randint(-1, 2), 0, width - 1)
                    self.samples[rand_i, rand_j, rand_sample] = frame[i, j]
        return fg_mask

cap = cv2.VideoCapture('video.mp4')
ret, frame = cap.read()
vibe = ViBe()
vibe.initialize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fg_mask = vibe.update(gray)
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask', fg_mask)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
